In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Wed Nov 20 11:53:55 PST 2019
------------

Distributor ID:	Ubuntu
Description:	Ubuntu 16.04.6 LTS
Release:	16.04
Codename:	xenial

------------
HOSTNAME: 
swoose

------------
Computer Specs:

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                24
On-line CPU(s) list:   0-23
Thread(s) per core:    2
Core(s) per socket:    6
Socket(s):             2
NUMA node(s):          1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 44
Model name:            Intel(R) Xeon(R) CPU           X5670  @ 2.93GHz
Stepping:              2
CPU MHz:               2925.971
BogoMIPS:              5851.97
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              12288K
NUMA node0 CPU(s):     0-23
Flags:                 fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr

No LSB modules are available.


### Set variables
`%env` variables are good for passing to bash cells

In [9]:
# Set workding directory
%env wd=/home/sam/analyses/20191120_swoose_pgen_v074_renaming
wd="/home/sam/analyses/20191120_swoose_pgen_v074_renaming"

%env old_fa_index=Pgenerosa_v074.fa.fai
%env base_cov_url=https://gannet.fish.washington.edu/seashell/bu-mox/scrubbed

env: wd=/home/sam/analyses/20191120_swoose_pgen_v074_renaming
env: old_fa_index=Pgenerosa_v074.fa.fai
env: base_cov_url=https://gannet.fish.washington.edu/seashell/bu-mox/scrubbed


In [3]:
%%bash
echo ${wget_options}

--recursive --accept deduplicated.bismark.cov.gz --no-parent --no-directories --quietly --no-check-certificate


#### Create necessary directories

In [4]:
%%bash
mkdir --parents ${wd}

In [5]:
cd {wd}

/home/sam/analyses/20191120_swoose_pgen_v074_renaming


#### Download Pgen_v074 files


In [12]:
%%bash
url_array=("${base_cov_url}"/0807/ "${base_cov_url}"/0807-003/ "${base_cov_url}"/0807-004/ "${base_cov_url}"/0809-005/)

# Downlaod Bismark coverage files
for url in "${url_array[@]}"
do
    echo "Downloading coverage files from ${url}"
    
    wget \
    --recursive \
    --accept deduplicated.bismark.cov.gz \
    --no-parent \
    --no-directories \
    --quiet \
    --no-check-certificate \
    "${url}"
done


# Remove index files
rm *.html*

# Download Pgen_074 FastA Index file
wget --quiet http://owl.fish.washington.edu/halfshell/genomic-databank/Pgenerosa_v074.fa.fai
    
ls -lh

total 2.5G
-rw-rw-r-- 1 sam sam  57M Aug 12 14:29 EPI-103_S27_L005_R1_001_val_1_bismark_bt2_pe.deduplicated.bismark.cov.gz
-rw-rw-r-- 1 sam sam  68M Aug 12 14:57 EPI-104_S28_L005_R1_001_val_1_bismark_bt2_pe.deduplicated.bismark.cov.gz
-rw-rw-r-- 1 sam sam  64M Aug 12 15:24 EPI-111_S29_L005_R1_001_val_1_bismark_bt2_pe.deduplicated.bismark.cov.gz
-rw-rw-r-- 1 sam sam  62M Aug 12 15:49 EPI-113_S30_L005_R1_001_val_1_bismark_bt2_pe.deduplicated.bismark.cov.gz
-rw-rw-r-- 1 sam sam  69M Aug 12 16:18 EPI-119_S31_L005_R1_001_val_1_bismark_bt2_pe.deduplicated.bismark.cov.gz
-rw-rw-r-- 1 sam sam  62M Aug 12 16:43 EPI-120_S32_L005_R1_001_val_1_bismark_bt2_pe.deduplicated.bismark.cov.gz
-rw-rw-r-- 1 sam sam  58M Aug 12 17:05 EPI-127_S33_L005_R1_001_val_1_bismark_bt2_pe.deduplicated.bismark.cov.gz
-rw-rw-r-- 1 sam sam  61M Aug 12 17:29 EPI-128_S34_L005_R1_001_val_1_bismark_bt2_pe.deduplicated.bismark.cov.gz
-rw-rw-r-- 1 sam sam  61M Aug 12 17:55 EPI-135_S35_L005_R1_001_val_1_bismark_bt2_pe.deduplica

rm: cannot remove '*.html*': No such file or directory


In [13]:
%%bash

zcat EPI-154_S5_L002_R1_001_val_1_bismark_bt2_pe.deduplicated.bismark.cov.gz | head

PGA_scaffold1__77_contigs__length_89643857	54	54	0	0	1
PGA_scaffold1__77_contigs__length_89643857	72	72	0	0	1
PGA_scaffold1__77_contigs__length_89643857	96	96	0	0	1
PGA_scaffold1__77_contigs__length_89643857	119	119	0	0	1
PGA_scaffold1__77_contigs__length_89643857	202	202	0	0	1
PGA_scaffold1__77_contigs__length_89643857	209	209	0	0	1
PGA_scaffold1__77_contigs__length_89643857	213	213	0	0	1
PGA_scaffold1__77_contigs__length_89643857	221	221	0	0	1
PGA_scaffold1__77_contigs__length_89643857	238	238	0	0	1
PGA_scaffold1__77_contigs__length_89643857	1254	1254	0	0	1


### Change scaffold names

In [ ]:
%%bash
# Array of old scaffold names
# Uses first column of FastA index file to get scaffold names
mapfile -t orig_scaffold_names < <(awk '{print $1}' "${old_fa_index}")

# Array of new scaffold names
mapfile -t new_scaffold_names < <(for number in {01..18}; do echo "Scaffold_${number}"; done)


for gzfile in *cov.gz
do
  # Save gunzipped filename (via parameter substitution - removes .gz from input filename)
  name="${gzfile%.*}"

  # Gunzip the gzipped file
  echo ""
  echo "Decompressing ${gzfile}"
  echo "Generating ${name}"
  gunzip "${gzfile}"


  # sed substitution
  # creates sed script to find original scaffold names and replace them with new scafold names
  # and passes to sed via stdin
  for index in "${!orig_scaffold_names[@]}"
    do
    printf "s/%s/%s/\n" "${orig_scaffold_names[index]}" "${new_scaffold_names[index]}"
  done | sed --file - "${name}" \
  >> "${name}".tmp

  # Remove the gunzipped file, now that a .tmp file exists
  echo ""
  echo "Now removing ${name}"
  rm "${name}"

  # Compress .tmp coverage files
  echo ""
  echo "Compressing ${name}.tmp to ${name}.gz"
  gzip --to-stdout "${name}".tmp > "${name}".gz
done

ls -ltrh

In [ ]:
%%bash
zcat EPI-154_S5_L002_R1_001_val_1_bismark_bt2_pe.deduplicated.bismark.cov.gz | head
zcat EPI-154_S5_L002_R1_001_val_1_bismark_bt2_pe.deduplicated.bismark.cov.gz | tail

# Clean up

In [ ]:
%%bash
rm *.tmp "${old_fa_index}"

ls -lh